In [3]:
#| default_exp routes.page

In [4]:
#| export
import httpx
import pandas as pd

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


In [5]:
#| export
async def get_page_by_id(auth: dmda.DomoAuth, page_id: str,
                         debug_api: bool = False, 
                         session: httpx.AsyncClient = None,
                         include_layout: bool = False
                         ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/content/v3/stacks/{page_id}/cards'
    
    if include_layout:
        url+='?includeV4PageLayouts=true'
    
    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session = session,
    )

    res.response


    return res

#### sample get_page_by_id

In [21]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
res = await get_page_by_id(page_id=page_id, auth=token_auth)

from pprint import pprint
pprint(res.response)


{'cards': [{'active': True,
            'allowTableDrill': True,
            'badgeUpdated': 1631619767000,
            'created': 1631619158,
            'creatorId': 1898323170,
            'description': '',
            'id': 1548948000,
            'locked': False,
            'metadata': {'SummaryNumberFormat': '{"type":"number","format":"#A"}',
                         'calendar': 'default',
                         'chartType': 'badge_vert_multibar',
                         'chartVersion': '8',
                         'columnAliases': '{}',
                         'columnFormats': '{}',
                         'currentLabel': '',
                         'currentMethod': 'empty',
                         'historyId': '6fa216c4-1465-4242-917c-3ca9e018c8be'},
            'ownerId': 1898323170,
            'title': 'View of 75th Percentile Test - Baseball Stats',
            'type': 'kpi',
            'urn': '1548948000'},
           {'active': True,
            'allowTableDril

In [22]:
# | export
async def get_page_definition(auth, page_id, debug_api: bool = False, session: httpx.AsyncClient = None):

    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/stacks/{page_id}/cards"

    params = {"includeV4PageLayouts": "true",
              "parts": "metadata,datasources,library,drillPathURNs,certification,owners,dateInfo,subscriptions,slicers"}

    res = await gd.get_data(url,
                            method='GET',
                            auth=auth,
                            session=session,
                            params=params, debug_api=debug_api)

    return res


#### sample implementation of get_page_definition

In [23]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
res = await get_page_definition(page_id=page_id, auth=token_auth)

pprint(res.response)


{'cards': [{'active': True,
            'allowTableDrill': True,
            'badgeUpdated': 1631619767000,
            'certification': {'adminCertified': False,
                              'state': 'NOT_CERTIFIED'},
            'created': 1631619158,
            'creatorId': 1898323170,
            'datasources': [{'adc': False,
                             'dataSourceId': '4ef43af5-67e7-4b9e-bd58-c4e592aa289a',
                             'dataSourceName': 'View of 75th Percentile Test - '
                                               'Baseball Stats',
                             'dataType': 'dataset-view',
                             'displayType': 'dataset-view',
                             'isSampleData': False,
                             'lastUpdated': 1673375782860,
                             'phase': None,
                             'providerType': 'dataset-view',
                             'state': 'IDLE'}],
            'dateInfo': {},
            'description'

In [24]:
# | export
async def get_page_access_list(auth,
                              page_id,
                              is_expand_users: bool = True,
                              debug_api: bool = False, session: httpx.AsyncClient = None):
    """retrieves accesslist, which users and groups a page is shared with"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/share/accesslist/page/{page_id}?expandUsers={is_expand_users}"

    res = await gd.get_data(url,
                            method='GET',
                            auth=auth,
                            session=session, debug_api=debug_api)

    # add group members to users response
    if is_expand_users:
        group_users = [user for group in res.response.get(
            'groups') for user in group.get('users')]
        users = res.response.get('users') + group_users
        res.response.update({'users': users})

    return res


#### sample get_page_accesslist

In [25]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
res = await get_page_access_list(page_id=page_id, auth=token_auth, is_expand_users=True)

# for user in res.response.get('users'):
#     print({user.get('displayName')})

pprint(res.response)

{'groups': [{'active': True,
             'creatorId': 1893952720,
             'default': False,
             'hidden': False,
             'id': 746771171,
             'name': 'test creator',
             'type': 'adHoc',
             'users': [{'accepted': True,
                        'active': True,
                        'anonymous': False,
                        'created': 1618579051,
                        'department': '',
                        'displayName': 'Dean',
                        'emailAddress': 'dean.pangelinan@bluemoondigital.co',
                        'id': 1416802348,
                        'invitorUserId': 1893952720,
                        'modified': 1619022930265,
                        'pending': False,
                        'phoneNumber': '+1 (510) 508-7344',
                        'rights': 31,
                        'role': 'Privileged',
                        'roleId': 2,
                        'systemUser': False,
                     

In [26]:
# |export
async def get_pages_adminsummary(auth: dmda.DomoAuth,
                                 debug_loop: bool = False,
                                 debug_api : bool = False,
                                 limit = 35,
                                 session: httpx.AsyncClient = None):
    """retrieves all pages in instance user is able to see (but may not have been explicitly shared)"""

    url = f'https://{auth.domo_instance}.domo.com/api/content/v1/pages/adminsummary'

    offset_params = {
        'offset': 'skip',
        'limit': 'limit',
    }

    body = {"orderBy": "pageTitle", "ascending": True}

    def arr_fn(res) -> list[dict]:
        return res.response.get('pageAdminSummaries')

    res = await gd.looper(auth=auth,
                          method='POST',
                          url=url,
                          arr_fn=arr_fn,
                          offset_params=offset_params,
                          session=session,
                          loop_until_end=True,
                          body = body,
                          limit = limit, 
                          debug_loop=debug_loop, debug_api= debug_api)
    return res


In [27]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await get_pages_adminsummary( auth=token_auth, debug_loop = False, debug_api = False)

ResponseGetData(status=200, response=[{'pageId': 1316566624, 'pageTitle': '20210623_TRAINING_DomoStats Activity Log App', 'parentPageId': 127044793, 'parentPageTitle': 'JaeW_AtOnyx', 'topPageId': 522373865, 'topPageTitle': 'Learn Domo', 'ownerId': 1893952720, 'ownerName': 'Jae Wilson1', 'owners': [{'id': 1893952720, 'type': 'USER', 'displayName': 'Jae Wilson1'}], 'locked': False, 'lastModified': 1624448913807, 'cardCount': 34}, {'pageId': 384424178, 'pageTitle': '75th Percentile Test', 'parentPageId': None, 'parentPageTitle': None, 'topPageId': None, 'topPageTitle': None, 'ownerId': 1898323170, 'ownerName': 'Creed Smith', 'owners': [{'id': 1898323170, 'type': 'USER', 'displayName': 'Creed Smith'}], 'locked': False, 'lastModified': 1681764847029, 'cardCount': 6}, {'pageId': 1113451501, 'pageTitle': 'Aditya Jain Dev', 'parentPageId': None, 'parentPageTitle': None, 'topPageId': None, 'topPageTitle': None, 'ownerId': None, 'ownerName': None, 'owners': [{'id': 1345737456, 'type': 'USER', 'd

In [8]:
# |export
async def update_page_layout(auth: dmda.DomoAuth, 
                            layout_id: str,
                             body: dict,
                         debug_api: bool = False):
    url = f'https://{auth.domo_instance}.domo.com/api/content/v4/pages/layouts/{layout_id}'


    res = await gd.get_data(
        auth=auth,
        url=url,
        body=body,
        method='PUT',
        debug_api=debug_api
    )

    if debug_api:
        print(res)

    return res

async def put_writelock(auth: dmda.DomoAuth, 
                        layout_id: str, 
                        user_id: str,
                        epoch_time: int,
                         debug_api: bool = False):
    url = f'https://{auth.domo_instance}.domo.com/api/content/v4/pages/layouts/{layout_id}/writelock'
    body= {
        "layoutId": layout_id,
        "lockHeartbeat": epoch_time,
        "lockTimestamp": epoch_time,
        "userId":user_id
    }
  
    res = await gd.get_data(
        auth=auth,
        url=url,
        body=body,
        method='PUT',
        debug_api=debug_api
    )

    if debug_api:
        print(res)

    return res

async def delete_writelock(auth: dmda.DomoAuth,
                           layout_id: str,
                         debug_api: bool = False):
    url = f'https://{auth.domo_instance}.domo.com/api/content/v4/pages/layouts/{layout_id}/writelock'
    res = await gd.get_data(
        auth=auth,
        url=url,
        method='DELETE',
        debug_api=debug_api
    )

    if debug_api:
        print(res)

    return res

In [28]:
#| hide
import nbdev
nbdev.nbdev_export()

Bad pipe message: %s [b'*\xc3\x07\x08\xe1\xcf\x86\xa4Je', b"\x13\x8eDH\xf2\xd5\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01", b'\x06\x01\x03\x03', b'\x03', b'']
Bad pipe message: %s [b'', b'\x02']
Bad pipe message: %s [b'\x05\x02\x06']
Bad pipe message: %s [b')T\xe2\xe9\x1d\x1fs\xc8\xa3\x8f\xb1\x92\x1a\x13\xd7X+\t\x00\x00\xa6\xc0,